## Model Training

In [105]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [86]:
data_frame = pd.read_csv('data\insurance.csv')
data_frame.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [87]:
data_frame.drop(labels=['region'],axis=1,inplace=True)

In [88]:
X = data_frame.drop(labels=['expenses'],axis=1)
Y = data_frame[['expenses']]

In [89]:
Y

,expenses
0,16884.92
1,1725.55
2,4449.46
3,21984.47
4,3866.86
...,...
1333,10600.55
1334,2205.98
1335,1629.83
1336,2007.95


In [90]:
X

,age,sex,bmi,children,smoker
0,19,female,27.9,0,yes
1,18,male,33.8,1,no
2,28,male,33.0,3,no
3,33,male,22.7,0,no
4,32,male,28.9,0,no
...,...,...,...,...,...
1333,50,male,31.0,3,no
1334,18,female,31.9,0,no
1335,18,female,36.9,0,no
1336,21,female,25.8,0,no


In [91]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [92]:
print('catagorical col:',categorical_cols)
print('numerical col:',numerical_cols)

catagorical col: Index(['sex', 'smoker'], dtype='object')
numerical col: Index(['age', 'bmi', 'children'], dtype='object')


In [52]:
# df['smoker'] = df['smoker'].replace({'yes': 1, 'no': 0})
# df['sex'] = df['sex'].replace({'male': 1, 'female': 0})

In [93]:
df.head(3)

,age,sex,bmi,children,smoker,expenses
0,19,0,27.9,0,1,16884.92
1,18,1,33.8,1,0,1725.55
2,28,1,33.0,3,0,4449.46


In [96]:
from sklearn.impute import SimpleImputer # Handling missing values
from sklearn.preprocessing import StandardScaler # Handing feature scaling
from sklearn.preprocessing import OrdinalEncoder # for encoding

#pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [97]:
sex_catagory = ['male','female']
smoker_catagory = ['yes','no']

In [98]:
#numerical pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

#Catagorical pipeline
cat_pipeline = Pipeline(
    steps= [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[sex_catagory,smoker_catagory])),
        ('scaler',StandardScaler())

    ]
)

# Combine both the pipeline
preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

In [99]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2,random_state=30)

In [100]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [102]:
X_train.head()

,num_pipeline__age,num_pipeline__bmi,num_pipeline__children,cat_pipeline__sex,cat_pipeline__smoker
0,0.365697,1.047059,-0.091139,1.005623,0.505835
1,-1.200055,0.173714,-0.924633,-0.994408,0.505835
2,-0.630691,0.157236,1.575849,-0.994408,0.505835
3,1.646766,1.030581,-0.091139,1.005623,0.505835
4,-0.915373,-0.238241,-0.091139,-0.994408,0.505835


In [119]:
# Model training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [120]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [121]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
    'Support_vector_regressor': SVR(C=1,epsilon=0.2),
    'Random_forest_Regressor' : RandomForestRegressor(),
    'Adaboost': AdaBoostRegressor()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

    #Make Predictions
    Y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(Y_test,Y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 6086.831447852395
MAE: 4187.1843945769515
R2 score 75.98133683491785


Lasso
Model Training Performance
RMSE: 6086.764147879808
MAE: 4186.837557919214
R2 score 75.98186796394621


Ridge
Model Training Performance
RMSE: 6087.010501649047
MAE: 4187.662719955859
R2 score 75.97992372002483


Elasticnet
Model Training Performance
RMSE: 7125.704418169045
MAE: 5023.797012525465
R2 score 67.08287573472312


Support_vector_regressor
Model Training Performance
RMSE: 13212.856457021428
MAE: 8444.779853259994
R2 score -13.177400317949406


Random_forest_Regressor
Model Training Performance
RMSE: 5360.407325338616
MAE: 2967.3923082960196
R2 score 81.37218722662595


Adaboost
Model Training Performance
RMSE: 5110.957675712586
MAE: 3983.852826074197
R2 score 83.06555962491308


